In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import os
from PIL import Image
from pdf2image import convert_from_path
import shutil

In [ ]:
IMG_DIR = os.getcwd()
DIFMAP = f"{IMG_DIR}/difmap_images"
EHT_IMAGING = f"{IMG_DIR}/eht-imaging_images"
SMILI = f"{IMG_DIR}/smili_images"

# Helps with renaming convention
d_prefix = f"difmap_"
e_prefix = f"eht-imaging_"
s_prefix = f"smili_"
RENAMED = False
MOVED = False

In [ ]:
# Reshapes each image and adds white padding around the border
def img_reshape(img):
    img = Image.open(img).convert('RGB')
    img = img.resize((321, 275))
    img = np.asarray(img)
    img = np.pad(img, ((5,5), (5,5), (0,0)), mode='constant', constant_values=255)
    return img

In [ ]:
# Creates an image grid out of the image array
def image_grid(array, ncols=4):
    index, height, width, channels = array.shape
    nrows = 3
    
    img_grid = (array.reshape(nrows, ncols, height, width, channels)
              .swapaxes(1,2)
              .reshape(height*nrows, width*ncols, channels))
        
    return img_grid

In [ ]:
# Renames images to appropriate names
if RENAMED == False:
    for f in os.listdir(DIFMAP):
        if f.endswith('.pdf'):
            new = DIFMAP+'/'+d_prefix+f
            os.rename(DIFMAP+'/'+f, new)
            shutil.move(new, new[-118:])
            # print(new[-118:])

    for f in os.listdir(EHT_IMAGING):
        if f.endswith('.pdf'):
            new = EHT_IMAGING+'/'+e_prefix+f
            os.rename(EHT_IMAGING+'/'+f, new)
            shutil.move(new, new[-42:])
            # print(len(e_prefix+f))

    for f in os.listdir(SMILI):
        if f.endswith('.pdf'):
            new = SMILI+'/'+s_prefix+f
            os.rename(SMILI+'/'+f, new)
            shutil.move(new, new[-39:])
            # print(len(s_prefix+f))
    RENAMED = True

# Convert .pdf images into .jpg images
pdfs = [f for f in os.listdir(IMG_DIR) if f.endswith('.pdf')]

for i in pdfs:
    image = convert_from_path(f'{IMG_DIR}/{i}')
    image[0].save(f'{IMG_DIR}/{i[:-4]}.png', 'PNG')
    
# Finds all pngs and sorts them in alpha-numerical order - helps with plotting later
pngs = [f for f in os.listdir(IMG_DIR) if f.endswith('.png')]

pngs = sorted(pngs)

# Removes the final image grid if rerunning this cell again
if 'final_comparison.png' in pngs:
    pngs.remove('final_comparison.png')
    
    
# Creates an image array of all the new png images
img_arr = []

for image in pngs:
    img_arr.append(img_reshape(image))

img_arr = np.array(img_arr)

In [ ]:
result = image_grid(img_arr)
fig = plt.figure(figsize=(20., 30.), )
plt.axis('off')

plt.text(-30, 180, "DIFMAP", fontsize=23, rotation=90)
plt.text(-30, 500, "eht-imaging", fontsize=23, rotation=90)
plt.text(-30, 740, "SMILI", fontsize=23, rotation=90)

plt.text(95, -5, "April 5", fontsize=23)
plt.text(425, -5, "April 6", fontsize=23)
plt.text(745, -5, "April 10", fontsize=23)
plt.text(1085, -5, "April 11", fontsize=23)

plt.imshow(result)
plt.savefig("final_comparison.png", bbox_inches='tight')

In [ ]:
if MOVED == False:
    # Moves all pdfs and all new pngs back to original folder
    for f in os.listdir(IMG_DIR):
        if f.endswith('.pdf') or f.endswith('.png'):
            if d_prefix in f:
                shutil.move(f, DIFMAP+'/'+f)
            if e_prefix in f:
                shutil.move(f, EHT_IMAGING+'/'+f)
            if s_prefix in f:
                shutil.move(f, SMILI+'/'+f)
    MOVED = True
